In [ ]:
import os
try:
    os.chdir('/content')
    !git clone https://github.com/billyhjm/PHM_Education.git
    os.chdir('/content/PHM_Education')
except:
    pass

# Feature Practice - CWRU

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import hilbert
import z_dependancy as z_dep
plt.rc('font', size=20)

# 데이터 로드

## 회전 주파수 알아보기

In [ ]:
F_shaft_rpm = 1797
F_shaft = F_shaft_rpm/60
print('Shaft Frequency is', F_shaft)

## 베어링 주파수 DB 만들기

In [ ]:
F_BPFI = 5.4152
F_BPFO = 3.5848
F_FTF = 0.398
F_BSF = 4.7135

F_BPFI_w = F_BPFI*F_shaft
F_BPFO_w = F_BPFO*F_shaft
F_FTF_w = F_FTF*F_shaft
F_BSF_w = F_BSF*F_shaft

print('Outer Race Fault Frequency is', F_BPFO_w)

## 신호 로드하고 그려보기

In [ ]:
fs=12000

### 신호를 로드해 주세요. 
정상 데이터를 v_n, 결함 데이터를 v_f로 지정해 주세요

### 정상, 고장 신호를 한 그래프에 그려주세요

## 주파수 분석해보기

### 정상, 고장 신호에 대해 주파수 분석을 한 후 한 그래프에 그려주세요

### 정상, 고장 신호에 대해 필터링을 수행해 주세요.

In [ ]:
ftype='band'
Wn=np.array([2500,4500])
print(Wn)

정상신호에 대한 결과는 v_filter_n, 결함신호에 대한 결과는 v_filter_f 라는 변수로 지정해 주세요.

### 정상, 고장 신호에 대해 주파수 분석을 한 후 한 그래프에 그려주세요

## 필터링 포락선 분석해보기

### 정상, 고장 신호의 필터링 결과에 대해 포락선 분석을 수행해 주세요.
정상신호에 대한 결과는 v_filter_env_n, 결함신호에 대한 결과는 v_filter_env_f 라는 변수로 지정해 주세요.

In [ ]:
[f_filter_env_n, A_filter_env_n] = z_dep.z_fft_simple(v_filter_env_n-np.mean(v_filter_env_n),fs);
[f_filter_env_f, A_filter_env_f] = z_dep.z_fft_simple(v_filter_env_f-np.mean(v_filter_env_f),fs);

In [ ]:
plt.figure()
plt.plot(f_filter_env_f,A_filter_env_f,'r')
plt.plot(f_filter_env_n,A_filter_env_n,'b')
plt.xlabel('Frequency')
plt.ylabel('|Y|')
plt.axis([0,600,0,0.02])

for n in range(4):
    plt.axvline(n*F_shaft, color='k', linestyle='dashed')
    plt.axvline(n*F_BPFO_w, color='r', linestyle='dashed')

# Feature 추출

In [ ]:
band_energy_and_filter = [2000,4500]
band_enrgy_from_envelope = np.matrix([
                [F_BPFO_w*0.9, F_BPFO_w*1.1],
                [2*F_BPFO_w*0.9, 2*F_BPFO_w*1.1],
                [3*F_BPFO_w*0.9, 3*F_BPFO_w*1.1]
                ])

print(f'Band Energy and Filter Range: {band_energy_and_filter}')
print(f'Band Energy from Envelope: {band_enrgy_from_envelope}')

In [ ]:
feature_n, feature_name = z_dep.z_feature(v_n, fs, band_energy_and_filter, band_enrgy_from_envelope)
feature_f, feature_name = z_dep.z_feature(v_f, fs, band_energy_and_filter, band_enrgy_from_envelope)

plt.figure(figsize=(8,6))
plt.plot(feature_n,'-bo')
plt.plot(feature_f,'-rx')
plt.xticks(np.arange(0,np.size(feature_name)),feature_name,rotation='vertical')
plt.ylabel('Features')
plt.show()

In [ ]:
feature_ratio = feature_f/feature_n

plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.plot(feature_ratio,'-bo')
plt.xticks(np.arange(0,np.size(feature_name)),feature_name,rotation='vertical')
plt.axhline(y=1, color='r', linestyle='--')
plt.ylabel('Ratio')

plt.subplot(1,2,2)
plt.plot(feature_ratio,'-bo')
plt.xticks(np.arange(0,np.size(feature_name)),feature_name,rotation='vertical')
plt.axhline(y=1, color='r', linestyle='--')
plt.ylim([0,10])
plt.ylabel('Ratio')
plt.show()